## Lesson resources

* [https://youtu.be/Q0z-l2KRYFY Lesson video]
* [http://forums.fast.ai/t/lesson-7-discussion Forum discussion]
* [http://wiki.fast.ai/index.php/Lesson_7_Notes Lesson 7 notes]
* The notebooks:
** [https://github.com/fastai/courses/blob/master/deeplearning1/nbs/lesson6.ipynb Lesson 6] updated with pure python RNN and theano GRU details
** [https://github.com/fastai/courses/blob/master/deeplearning1/nbs/lesson7.ipynb Lesson 7] has the various CNN architectures applied to the kaggle fisheries competition.

* The python scripts:
** [https://github.com/fastai/courses/blob/master/deeplearning1/nbs/vgg16bn.py VGG with batch normalization] - also adds 'size' and 'include_top' parameters
** [https://github.com/fastai/courses/blob/master/deeplearning1/nbs/resnet50.py resnet50.py] - The resnet architecture

## More information

* [https://culurciello.github.io/tech/2016/06/04/nets.html Network architecture review] (really great)
* [http://techtalks.tv/talks/fully-convolutional-networks-for-semantic-segmentation/61606/ Fully convolutional net]
* [https://keras.io/getting-started/functional-api-guide/#multi-input-and-multi-output-models [Multi input and multi output nets in Keras]
* [http://www.wildml.com/2015/10/recurrent-neural-network-tutorial-part-4-implementing-a-grulstm-rnn-with-python-and-theano/ Gated recurrent units (GRUs) ]

## Fisheries competition

In this notebook we're going to investigate a range of different architectures for the [Kaggle fisheries competition]. The video states taht vgg.py and vgg_ft() from utils.py have been updated to include VGG with batch normalization, but this is not the case. We've instead created a new file [vgg_bn.py] and an additional method vgg_ft_bn() (which is already in utils.py) which we use in this notbook.

In [1]:
%matplotlib inline
import imp
import utils
imp.reload(utils)
from utils import *

Using TensorFlow backend.


In [2]:
path = 'data\\fish\\'
batch_size = 64

In [ ]:
batches = get_batches(path + 'train', batch_size = batch_sie)
val_batches = get_batches(path + 'valid', batch_size = batch_size * 2, shuffle = False)

(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(path)